# Count articles and split file

In [1]:
import pandas as pd
import csv

In [8]:
%%time
cnt = 0

for chunk in pd.read_csv('GeoWAC.rus.Sampled', encoding='utf-8', low_memory=True, chunksize=100000):
     cnt += 1
     if cnt % 50 == 0:
         print(f'Current -- {cnt * 100000:,}')

Current -- 5,000,000
Current -- 10,000,000
Current -- 15,000,000
Current -- 20,000,000
Current -- 25,000,000
Current -- 30,000,000
Current -- 35,000,000
Current -- 40,000,000
Wall time: 6min 57s


In [5]:
%%time
cnt = 0

for chunk in pd.read_csv('text.csv', encoding='utf-8', low_memory=True, chunksize=100000, header=None, names = ['Text']):
    chunk = chunk.dropna()
    chunk = chunk[chunk['Text'] != 'EOS']
    chunk = chunk[chunk['Text'] != 'N_A']
    cnt += len(chunk)
    
print(f'Articles: {cnt}')

Articles: 8152686
Wall time: 2min 10s


In [9]:
print(f'Total chunks -- {cnt}')
print(f'Total lines -- {cnt * 100000:,}')

Total chunks -- 429
Total lines -- 42,900,000


In [9]:
%%time

i = 0
cnt = 0
file_name = 'geo_wac0.csv'
cs = open(file_name, 'w', newline='', encoding='utf-8')
linewriter = csv.writer(cs)

for chunk in pd.read_csv('GeoWAC.rus.Sampled', encoding='utf-8', low_memory=True, chunksize=100000):
    i += 100000
    tmp = chunk.Text.to_list()
    tmp = [[t] for t in tmp]
    linewriter.writerows(tmp)

    if i == 5000000:  
        cs.close() 
        i = 0          
        cnt += 1
        print('Starting new file')
        file_name = 'geo_wac' + str(cnt) + '.csv'
        cs = open(file_name, 'w', newline='', encoding='utf-8')
        linewriter = csv.writer(cs)
linewriter.writerows(tmp)    

Starting new file
Starting new file
Starting new file
Starting new file
Starting new file
Starting new file
Starting new file
Starting new file
Wall time: 14min 56s


In [10]:
cs.close() 

In [ ]:
%%time

i = 0
cnt = 0
# file_name = 'texts0.csv'
# cs = open(file_name, 'w', newline='', encoding='utf-8')
linewriter = csv.writer(cs)

for chunk in pd.read_csv('text.csv', encoding='utf-8', low_memory=True,
                         chunksize=100000, header=None, names = ['Text']):
    i += 100000
    tmp = chunk.Text.to_list()
    tmp = [process(t) for t in tmp]
    linewriter.writerows(tmp)

    if i == 5000000:  
        cs.close() 
        i = 0          
        cnt += 1
        print('Starting new file')
        file_name = 'texts' + str(cnt) + '.csv'
        cs = open(file_name, 'w', newline='', encoding='utf-8')
        linewriter = csv.writer(cs)
linewriter.writerows(tmp)    

### Clean file from unnecessary blank lines

In [2]:
%%time
import re

def read_in_chunks(file_object, chunk_size=50000):
    """Lazy function (generator) to read a file piece by piece.
    Default chunk size: 1k."""
    while True:
        data = file_object.read(chunk_size)
        if not data:
            break
        yield data

with open('geo_wac8_v_09_cleaned.txt', 'w', encoding='utf-8') as f1:
    with open('geo_wac8_v_09.txt', encoding='utf-8') as f:
        for piece in read_in_chunks(f):
            reps = re.sub('\n{3,}', '\n\n', piece)
            reps = re.sub('\n\n \n', '\n\n', reps)
            f1.write(reps)       

Wall time: 19.8 s


In [2]:
%%time
import pandas as pd

with open('wiki5.txt', 'w', encoding='utf-8') as f:
    for chunk in pd.read_csv('texts5.csv', header=None, names = ['Text'],
                             encoding='utf-8', low_memory=True, chunksize=10000):
        chunk = chunk.dropna()
        chunk = chunk.Text.to_list()
        for line in chunk:
            if line != 'EOS':
                line += '\n'
                f.write(line)
            else:
                f.write('\n')

Wall time: 55.7 s


# Clean RNC

In [9]:
%%time
import re

def read_in_chunks_lines(file_object, chunk_size=50000):
    """Lazy function (generator) to read a file piece by piece.
    Default chunk size: 1k."""
    while True:
        data = file_object.readlines(chunk_size)
        if not data:
            break
        yield data
        
def clean(x):
    x = re.sub('(?<!.)\W+?\n(?!.)', '\n', x)
    return x

with open('rnc_post-soviet_cleaned.txt', 'w', encoding='utf-8') as f1:
    with open('rnc_post-soviet.txt', encoding='utf-8') as f:
        for piece in read_in_chunks_lines(f):
            reps = [clean(x) for x in piece]
            for y in reps:
                f1.write(y)

Wall time: 33.4 s


In [10]:
with open('rnc_post-soviet_cleaned.txt', encoding='utf-8') as f:
    print(f.readlines(1000))

['Занимается аэробикой и верховой ездой, хорошо играет на гитаре, поет.\n', 'И я стал размышлять: "Может быть, пока, временно, стоит поучиться в каком-нибудь другом институте" Но в каком Я был еще очень зелен, не готов к выбору.\n', 'Я делаю еще два круга.\n', '-- Понял! -- заговорщицки кивнул мастер.\n', 'Являемся на первую квартиру.\n', 'Возглавлявший комиссию Михаил Доримедонтович Быковский (1801-1885) учился у Д. И. Жилярди, с 1828 года -- архитектор, с 1831 года -- академик архитектуры, с 1834 года служил чиновником особых поручений при московском генерал-губернаторе, вместе с Е.\n', 'Я рассмеялась.\n', 'Он свой трактор сломал, вот и орет.\n', 'Говоря о развитии межрелигиозного диалога в пределах СНГ, хотелось бы особо отметить прошедшие в Москве в ноябре 2000 года и в марте 2004 года I и II Межрелигиозные миротворческие форумы, по сути ставшие первыми саммитами духовных лидеров стран Содружества в постсоветский период.\n', '-- Да я что, я с дорогой душой, -- проговорил Скосарев.\

In [11]:
%%time
import re

def read_in_chunks(file_object, chunk_size=50000):
    """Lazy function (generator) to read a file piece by piece.
    Default chunk size: 1k."""
    while True:
        data = file_object.read(chunk_size)
        if not data:
            break
        yield data

with open('rnc_post-soviet_cleaned1.txt', 'w', encoding='utf-8') as f1:
    with open('rnc_post-soviet_cleaned.txt', encoding='utf-8') as f:
        for piece in read_in_chunks(f):
            reps = re.sub('\n{3,}', '\n\n', piece)
            reps = re.sub('\n\n \n', '\n\n', reps)
            f1.write(reps)       

Wall time: 14.2 s


# Count sentences before/after deduplication

In [1]:
from spacy.lang.ru import Russian
nlp = Russian()
nlp.add_pipe('sentencizer')

In [3]:
rus = Russian()
tokenizer = rus.tokenizer

In [2]:
def process(section):
    res = []
    doc = nlp(section)
    for sent in doc.sents:
        sent = str(sent)
        x = sent.replace('\n', '')
        if x:
            res.append([x])
    return res

In [4]:
%%time
import pandas as pd

words = 0
sents = 0
words_wo_punct = 0
words_wo_stops = 0
words_wo_stops_punct = 0

def read_in_chunks_lines(file_object, chunk_size=50000):
    """Lazy function (generator) to read a file piece by piece.
    Default chunk size: 1k."""
    while True:
        data = file_object.readlines(chunk_size)
        if not data:
            break
        yield data

for chunk in pd.read_csv('Исходники\geo_wac0.csv', header=None, names = ['Text'], encoding='utf-8',
                             low_memory=True, chunksize=500000):
    chunk = chunk.dropna()
    df_geo = chunk.Text.to_list()
    geo_sents = []    
    
    for text in df_geo:
        sent = process(text)
        geo_sents.extend(sent)
        
    geo_sents = [item for sublist in geo_sents for item in sublist]
    sents += len(geo_sents)
    
    for piece in geo_sents:
        if piece:
            doc = tokenizer(piece)
            words += len([w.text for w in doc])
            words_wo_punct += len([token for token in doc if not token.is_punct])
            words_wo_stops += len([token.text for token in doc if not token.is_stop])
            words_wo_stops_punct += len([token for token in doc if not token.is_stop and not token.is_punct])
        
print(f'Subtotal: {sents} sentences before deduplication')
print(f'Subtotal: {words} words before deduplication')  
print(f'Subtotal: {words_wo_punct} tokens without punctuation')
print(f'Subtotal: {words_wo_stops} tokens without stops')
print(f'Subtotal: {words_wo_stops_punct} tokens without stops and punctuation') 

Subtotal: 16071469 sentences before deduplication
Subtotal: 280665998 words before deduplication
Subtotal: 233591819 tokens without punctuation
Subtotal: 217885182 tokens without stops
Subtotal: 170811003 tokens without stops and punctuation
Wall time: 31min 24s


In [5]:
del chunk
del geo_sents
del df_geo

In [6]:
def process_txt(x):
    if x != '\n':
        x = x.replace('\n', '')
        return x

In [ ]:
def read_in_chunks_lines(file_object, chunk_size=5000000):
    while True:
        data = file_object.readlines(chunk_size)
        if not data:
            break
        yield data

In [15]:
%%time
words = 0
sents = 0
words_wo_punct = 0
words_wo_stops = 0
words_wo_stops_punct = 0

with open('Ready\geo_wac8_v_09_cleaned.txt', encoding='utf-8') as f1:
        for piece in read_in_chunks_lines(f1):
            geo_sents = [process_txt(x) for x in piece if x != '\n']            
            sents += len(geo_sents)

            for piece in geo_sents:
                if piece:
                    doc = tokenizer(piece)
                    words += len([w.text for w in doc])
                    words_wo_punct += len([token for token in doc if not token.is_punct])
                    words_wo_stops += len([token.text for token in doc if not token.is_stop])
                    words_wo_stops_punct += len([token for token in doc if not token.is_stop and not token.is_punct])

print(f'Subtotal: {sents} sentences after deduplication')
print(f'Subtotal: {words} words after deduplication')  
print(f'Subtotal: {words_wo_punct} tokens without punctuation')
print(f'Subtotal: {words_wo_stops} tokens without stops')
print(f'Subtotal: {words_wo_stops_punct} tokens without stops and punctuation')

Subtotal: 8353796 sentences after deduplication
Subtotal: 163549838 words after deduplication
Subtotal: 137979529 tokens without punctuation
Subtotal: 126619384 tokens without stops
Subtotal: 101049075 tokens without stops and punctuation
Wall time: 7min 39s
